# Benchmarking annotation transfer

**This code will allow benchmarking of various annotation transfer tools including :**

- Liftoff
- RGAAT
- RATT
- Companion (based on RATT)
- Segment_liftover

**Genomes to be compared are as follows:**

In [ ]:
aspergillus_genomes = {"Aspergillus aculeatus ATCC 16872":"GCA_001890905.1",
"Aspergillus brasiliensis CBS 101740":"GCA_001889945.1",
"Aspergillus campestris IBT 28561":"GCA_002847485.1",
"Aspergillus carbonarius ITEM 5010":"GCA_001990825.1",
"Aspergillus clavatus NRRL 1":"GCA_000002715.1",
"Aspergillus cristatus GZAAS20.1005":"GCA_001717485.1",
"Aspergillus eucalypticola CBS 122712":"GCA_003184535.1",
"Aspergillus fijiensis CBS 313.89":"GCA_003184825.1",
"Aspergillus fischeri NRRL 181":"GCA_000149645.4,
"Aspergillus flavus NRRL3357":"GCA_000006275.3",
"Aspergillus flavus NRRL3357 2020":"GCA_009017415.1",
"Aspergillus fumigatus A1163":"GCA_000150145.1",
"Aspergillus fumigatus Af293":"GCA_000002655.1",
"Aspergillus glaucus CBS 516.65":"GCA_001890805.1",
"Aspergillus heteromorphus CBS 117.55":"GCA_003184545.1",
"Aspergillus lentulus strain IFM 54703":"GCA_001445615.2",
"Aspergillus luchuensis IFO 4308":"GCA_000239835.2",
"Aspergillus nidulans FGSC A4":"GCA_000149205.2",
"Aspergillus niger ATCC 1015":"GCA_000230395.2",
"Aspergillus niger ATCC 13496":"GCA_003344705.1",
"Aspergillus niger CBS 513.88":"GCA_000002855.2",
"Aspergillus niger strain N402 (ATCC64974)":"GCA_900248155.1",
"Aspergillus novofumigatus IBT 16806":"GCA_002847465.1",
"Aspergillus ochraceoroseus IBT 24754":"GCA_002846915.2",
"Aspergillus oryzae RIB40":"GCA_000184455.3",
"Aspergillus parasiticus CBS 117618":"GCA_009176385.1",
"Aspergillus steynii IBT 23096":"GCA_002849105.1",
"Aspergillus tanneri NIH1004":"GCA_004798825.1",
"Aspergillus terreus NIH2624":"GCA_000149615.1",
"Aspergillus thermomutatus strain HMR AF 39":"GCA_002237265.2",
"Aspergillus uvarum CBS 121591":"GCA_003184745.1",
"Aspergillus wentii DTO 134E9":"GCA_001890725.1"}

## Genome quality checks

First, quailty checks with ANI, AAI and BUSCO

In [ ]:
for seq1 in $(cat names)
do

for seq2 in $(cat names)
do
# run ANI 
# Konstantinidis & Tiedje, 2005, PNAS; Altschul et al, 2000, JMB (BLAST); Kent WJ, 2002, Genome Res (BLAT); Rodriguez-R & Konstantinidis, 2016, PeerJ Preprints. 
echo "$(tput setab 6)$(tput setaf 0)running ANI anaylsis for ${seq1} and ${seq2} ... $(tput sgr 0)"
ani.rb --seq1 ${seq1}_Genome.fasta --seq2 ${seq2}_Genome.fasta --tab temp
printf '${seq1}\t${seq2}\t$(cat temp)\n' >> ANI-list.tsv
done

for seq2 in $(cat names)
do
# run AAI 
# Konstantinidis & Tiedje, 2005, PNAS; Altschul et al, 2000, JMB (BLAST); Kent WJ, 2002, Genome Res (BLAT); Rodriguez-R & Konstantinidis, 2016, PeerJ Preprints. 
echo "$(tput setab 6)$(tput setaf 0)running ANI anaylsis for ${seq1} and ${seq2} ... $(tput sgr 0)"
aai.rb --seq1 ${seq1}_AnnotatedProteins.fasta --seq2 ${seq2}_AnnotatedProteins.fasta --tab temp
printf "${seq1}\t${seq2}\t$(cat temp)\n" >> AAI-list.tsv
done

    

In [ ]:
for seq1 in $(cat names)
do

# run BUSCO
# Manni M., Berkeley M.R., Seppey M., Simao F.A., Zdobnov E.M. 2021. BUSCO update: novel and streamlined workflows along with broader and deeper phylogenetic coverage for scoring of eukaryotic, prokaryotic, and viral genomes. arXiv:2106.11799 [q-bio] [Internet]. Available from: http://arxiv.org/abs/2106.11799
echo "$(tput setab 6)$(tput setaf 0)Running BUSCO analysis for ${i} ... $(tput sgr 0)"
busco -i ${seq1}_Genome.fasta -o ${seq1}-BUSCO-out -m genome --auto-lineage-euk

done

## Program tests

In [ ]:
# liftoff
for seq1 in $(cat names)
do
echo "$(tput setab 6)$(tput setaf 0)Running liftoff for ${seq1} ... $(tput sgr 0)"
mkdir ${seq1}-liftoff
cut -f3 ${seq1}.gff | sort | uniq | grep -v "#" > ${seq1}-liftoff/${seq1}-features
liftoff -g ${seq1}.gff -o ${seq1}-liftoff/${seq1}-lifted_annotations.gff3 -f ${seq1}-liftoff/${seq1}-features -u ${seq1}-liftoff/${seq1}-unmapped-features -polish ${seq1}_Genome.fasta ${seq1}_Genome.fasta
echo "$(tput setab 6)$(tput setaf 0)Completed liftoff for ${seq1} ... $(tput sgr 0)"
done

## Assessing the model transfers

*Notes*
TNR (true negative rate) is the same as SP (specificity), but in the context of AED (annotation edit distance), SP is the same as PPV (positive predictive value)
TPR (true positive rate) is the same as SN (sensitivity)


In [ ]:
import numpy as np
import gffutils

def read_gff_pair(og_gff,new_gff):
    # access the coding and none coding genes by location
    # make sure the annotations match between the original 
    # annotation and the newly transferred annotation
    fn1 = gffutils.example_filename(og_gff)
    fn2 = gffutils.example_filename(new_gff)

    db1 = gffutils.create_db(fn1, dbfn=str(og_gff)+".db", force=True)
    db2 = gffutils.create_db(fn2, dbfn=str(new_gff)+".db", force=True)

    fdb1 = gffutils.FeatureDB(str(og_gff)+".db", keep_order=True)
    fdb2 = gffutils.FeatureDB(str(new_gff)+".db", keep_order=True)

    # get gff features
    # start, stop, orientation, contig
    for i in fdb1.children(gene, featuretype=["protein coding gene", "none protein coding gene","pseudogene"], order_by='start'):
        start = i.start
        end = i.stop
        orientation = i.orientation



In [ ]:
import numpy as np
from math import sqrt

def basic_metrics(tally):
    tp, tn, fp, fn, _ = tally
    return{
        "TPR": tp/(tp+fn),
        "TNR": tn/(tn+fp),
        "PPV": tp/(tp+fp),
        "NPV": tn/(tn+fn),
        "FPR": fp/(fp+tn),
        "FNR": fn/(fn+tp)
    }

def advanced_metrics(tally, m):
    tp, tn, fp, fn, _ = tally
    n = tp+tn+fp+fn
    po = (tp+tn)/n # observed accuracy
    pe = (tp+fn)*(tp+fp)/n**2 + (tn+fp)*(tn+fn)/n**2 # accuracy expected by chance


    return {
        "F1": 2.0*m["PPV"]*m["TPR"]/(m["PPV"] + m["TPR"]), 
        "MCC": (tp*tn - fp*fn) / sqrt((tp+fp)*(tp+fn)*(tn+fp)*(tn+fn)),
        "kappa": (po - pe) / (1.0 - pe),
        "informedness": m["TPR"] + m["TNR"] - 1.0,
        "markedness": m["PPV"] + m["NPV"] - 1.0,
        "AED": 1 - ((m["PPV"] + m["TPR"])/2)   
        }


tally = compare_features(og_gff,new_gff)
m = basic_metrics(tally)
am = advanced_metrics(tally, m)